# SGP 2025 - Deep Learning on Meshes and Point Clouds tutorial

In this tutorial we give a quick introduction in the basics of deep learning on a point cloud. We will:
1. Set up the Python environment with the right libraries.
2. Load a dataset and visualize the contents.
3. Set up a deep learning model on this dataset.
4. Train the model and visualize its outputs.

During the tutorial, you will find some lines marked with $\rightarrow$.<br />
$\rightarrow$ Pause and try out the tasks get deeper understanding.

## Step 1: Set up the Python environment
In this Notebook, we'll need the following libraries:
- PyTorch - a general library for deep learning, accelerated on GPU.
- PyTorch Geometric (pyg) - an 'extension' library for learning on graphs and geometric data.
- Tensorboard - a framework to monitor training and visualize outputs.
- Polyscope - a convenient and extensible 3D viewer.
- Meshplot - a 3D viewer that can run inside Jupyter notebooks.

In [ ]:
try:
    import torch
    import torch_geometric as pyg
    import polyscope as ps
    import meshplot as mp
    from tqdm.notebook import tqdm
except ImportError:
    print("One of the dependencies is not installed. Installing now...")
    %pip install torch torchvision
    %pip install torch_geometric torch_cluster polyscope tqdm
    %pip install matplotlib pythreejs meshplot@git+https://github.com/skoch9/meshplot/@725e4a7926a5f10888f0edd1762fecf9db751c56

### Testing the environment
We want to make sure that all the libraries we installed work (we expect no errors when running the imports) and the CUDA components are available, if you have a GPU.

Do not worry if CUDA support says `False`, you can still run the tutorial on your CPU.

In [ ]:
print(f"CUDA support: {torch.cuda.is_available()}")

## Step 2: Load a dataset and visualize the contents.

We will use PyTorch Geometric's (pyg) built-in dataset classes to load the ModelNet10 dataset from the [“3D ShapeNets: A Deep Representation for Volumetric Shapes”](https://people.csail.mit.edu/khosla/papers/cvpr2015_wu.pdf) paper. If you'd like to make your own datasets, be sure to check out the [PyTorch Geometric tutorial on "Creating Graph Datasets"](https://pytorch-geometric.readthedocs.io/en/latest/tutorial/create_dataset.html).

We've created a smaller version of the dataset for this tutorial. Let's first download and unzip that, so PyTorch Geometric does not download the full dataset.

In [ ]:
import os

url_modelnet10 = "https://www.dropbox.com/scl/fi/x9pjg7orfv1f0gwcz3vux/ModelNet10.zip?rlkey=ji5zjyt52v8aipz0ecdhqqx7u&st=oem5rp7v&dl=1"
raw_dir = 'data/raw'
if not os.path.exists(raw_dir):
    os.makedirs(raw_dir)

path = pyg.data.download_url(url_modelnet10, raw_dir)
pyg.data.extract_zip(path, raw_dir)
os.unlink(path)
pyg.io.fs.rm(os.path.join(raw_dir, '__MACOSX'))

In [ ]:
dataset = pyg.datasets.ModelNet(root='data', name='10', train=True, force_reload=True)

Let's take a look at the dataset. The main thing you need to know for now is that you can access the elements in the dataset using Python's `[]` accessor.

In [ ]:
import os

# List all class names in the ModelNet10 dataset
class_names = [n for n in sorted(os.listdir(dataset.raw_dir)) if os.path.isdir(os.path.join(dataset.raw_dir, n))]

# Access one object in the dataset
# We retrieve a Data object, which contains the vertex positions, faces and a label.
print(f"Number of objects in the dataset: {len(dataset)}")
print(f"Number of classes: {dataset.num_classes}")
print(f"Class names: {class_names}\n")

print(f"Data object 0: {dataset[0]}")
print(f"Object 0 label: {class_names[dataset[0].y.item()]}")

It's good practice to visualize your data before and after processing and applying transformations (e.g., computing a neighborhood graph, normalizing scale). This lets you catch bugs or problems with the data early on.

We'll use Polyscope one time and proceed with `meshplot` in the rest of the notebook. Polyscope is great for interactive inspection and it lets you prototype tools and complex visualizations.

*Note 1: If you're on Mac, the Polyscope window may open in the background.*<br />
*Note 2: Close the Polyscope window to stop the process running in this notebook.*


In [ ]:
ps.init()
ps.register_surface_mesh("ModelNet Object", 
                         # We need to convert from torch to numpy for polyscope
                          dataset[0].pos.numpy(),
                                   # We transpose the face array, because polyscope expects faces to be in [F, 3] format 
                          dataset[0].face.T.numpy())
ps.show()

If you're working in the cloud on a Jupter notebook, it's easier to use `meshplot`, because it runs directly within Jupyter. We'll continue with `meshplot` in the remainder, so you can also follow along in a cloud hosted notebook.

$\rightarrow$ Try plotting a couple of different meshes.

In [ ]:
index = 0
mp.plot(dataset[index].pos.numpy(), dataset[index].face.T.numpy())

It's good to know where data resides in PyTorch. Tensors (the core data objects in PyTorch, i.e., n-dimensional arrays) can be stored near the CPU (RAM) or on the GPU (VRAM). Whenever you want to perform an action with the CPU or GPU, you need to make sure that the tensor is placed on the right device. 

**However, beware of going back and forth between CPU and GPU!** One of the biggest bottlenecks in GPU computing is the trip down memory lane, the literal one. It is preferrable to move data only once (although it's definitely possible to move back-and-forth). For example, you can first process your data on the CPU and move it to the GPU in batches.

Our data currently resides on the CPU, because we haven't told it to move to the GPU. If we want that to happen, we can use `tensor.to('cuda')`.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

pos = dataset[0].pos

print(f"Device for pos before moving to the GPU: {pos.device}")
pos = pos.to(device)
print(f"Device for pos after moving to the GPU: {pos.device} (remains 'cpu' if you do not have a GPU available)")

### Converting to a point cloud and then a graph
In the lecture for this course, we discussed the graph-based approach to deep learning on meshes and point clouds. We can transform the meshes in this dataset to a point cloud by sampling points on the faces.

$\rightarrow$ What would be the issue with directly using the mesh vertices? How can we account for those issues?

**Hint:** Look at a few of the meshes that we plotted before. What would happen if the triangles change, but the surface stays the same (e.g., you subdvide some triangles)?

#### Transforms
These processing steps are implemented in pyg as _transforms_. They are applied to all objects in the dataset and (a) stored on disk if you give it as a pre-transform or (b) applied on-the-go when loading the objects if given as a normal transform.

We setup the following pre-transforms:
- Normalize each mesh to a unit cube.
- Sample points on the faces of the mesh (removes faces and original vertices).
- Create a kNN graph for each point cloud.

We apply these as pre-transforms, because we would like to re-use these steps every time we train.

During training, we would like to apply a random rotation and scale, to make the method robust to these operations when testing. It's better if we do not use these as pre-transforms, because we want the rotation and scale to be different every time we load a shape. We add those as regular transforms, which are applied when an object is loaded during training.

**Note:** processing can take a while!

In [ ]:
import torch_geometric.transforms as T

# Transforms applied before loading the dataset to memory, 
# these are applied only once and not repeated during training
pre_transform = T.Compose((
    T.NormalizeScale(),
    T.SamplePoints(1024), # If we want to sample more points, we can change this number
    T.KNNGraph(k=20, loop=False)
))

# Transforms applied during loading, repeated during training
# We can use these to augment the data
# For example, we can randomly scale and rotate the objects
transform = T.Compose((
    T.RandomScale((0.85, 1.15)),
    T.RandomRotate(45, axis=0),
    T.RandomRotate(45, axis=1),
    T.RandomRotate(45, axis=2))
)

# We load the dataset with the pre-transforms and the transform
train_dataset = pyg.datasets.ModelNet(root='data', name='10', train=True, transform=transform, pre_transform=pre_transform, force_reload=True)
# And also the test dataset. Note that we do not apply the data augmentation transforms to the test set
test_dataset = pyg.datasets.ModelNet(root='data', name='10', train=False, transform=transform, pre_transform=pre_transform)

$\rightarrow$ Again, visualize the point clouds for several shapes.

In [ ]:
index = 0
mp.plot(train_dataset[index].pos.numpy(), shading={"point_size": 0.1})

# Alternatively, visualize the point cloud in polyscope
# ps.init()
# ps.register_point_cloud("ModelNet Object",
#                         train_dataset[index].pos.numpy(), radius=0.0002)
# ps.show()

The dataset lets you load one `Data` object at a time. What if you want to send multiple shapes at the same time (e.g., for Stochastic Gradient Descent)? In that case, we need to put a batch of shapes together.

This is handled by a `DataLoader`. The data loader also allows you to set a number of workers. You can think of this as the number of parallel processes to load and process your data from RAM. You can tweak this, so your CPU can 'catch up' with the GPU.

In [ ]:
batch_size = 32

# Create data loaders for the training and test datasets
# We let the train data loaders automatically batch the data and shuffle it for training with the `shuffle=True` argument
train_loader = pyg.loader.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = pyg.loader.DataLoader(test_dataset, batch_size=batch_size)

## Step 3: Set up a deep learning model on this dataset.

We will now setup a simple PointNet++ model using Message Passing. This code is based on the [Point Cloud Processing tutorial in the PyTorch Geometric documentation.](https://pytorch-geometric.readthedocs.io/en/latest/tutorial/point_cloud.html)

PyTorch works with Modules. Modules are objects that process and output data, often with some parameters that are stored in the object. This allows PyTorch to keep track of the parameters that need to be optimized. For example, in the code below, we create a `PointNetLayer`, which is a descendant of `torch.nn.Module` (through `MessagePassing`). The MLP used in PointNet is initialized when the `PointNetLayer` is constructed, so that the weights in the MLP can be optimized.

$\rightarrow$ Follow the comments in the code to understand what is going on.

In [ ]:
from torch import Tensor
from torch.nn import Sequential, Linear, ReLU

from torch_geometric.nn import MessagePassing


class PointNetLayer(MessagePassing):
    """We implement a PointNet layer based on the MessagePassing class from PyTorch Geometric.
    The MessagePassing class requires a function to compute the message
    and a forward function, that you can fill in with your own logic.

    Aggregation is handled by the MessagePassing class and only requires
    that you set the `aggr` argument in the constructor.
    """
    def __init__(self, in_channels: int, out_channels: int):
        # Message passing with "max" aggregation.
        # TODO: Experiment with other aggregations like "mean" or "add".
        super().__init__(aggr='max')

        # Initialization of the MLP used to compute messages:
        # Here, the number of input features correspond to the hidden
        # node dimensionality plus point dimensionality (=3).
        self.mlp = Sequential(
            Linear(in_channels + 3, out_channels),
            ReLU(),
            Linear(out_channels, out_channels),
        )

    def forward(self,
        h: Tensor,
        pos: Tensor,
        edge_index: Tensor,
    ) -> Tensor:
        # Start propagating messages.
        return self.propagate(edge_index, h=h, pos=pos)

    def message(self,
        h_i: Tensor,
        h_j: Tensor,
        pos_j: Tensor,
        pos_i: Tensor,
    ) -> Tensor:
        # h_j: The features of the central node as shape [num_edges, in_channels]
        # h_j: The features of neighbors as shape [num_edges, in_channels]
        # pos_j: The position of neighbors as shape [num_edges, 3]
        # pos_i: The central node position as shape [num_edges, 3]

        # TODO: Experiment with different edge features (e.g., EdgeConv, GCN)
        edge_feat = torch.cat([h_j, pos_j - pos_i], dim=-1)
        return self.mlp(edge_feat)

Once the main building block (the PointNetLayer) is created, we can stack them together into a simple PointNet++ architecture.

In [ ]:
from torch_geometric.nn import global_max_pool


class PointNet(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # We setup two PointNet layers and add a simple classifier at the end.
        self.conv1 = PointNetLayer(3, 32)
        self.conv2 = PointNetLayer(32, 32)
        self.classifier = Linear(32, dataset.num_classes)

    def forward(self,
        pos: Tensor,
        edge_index: Tensor,
        batch: Tensor,
    ) -> Tensor:

        # Perform two-layers of message passing:
        h = self.conv1(h=pos, pos=pos, edge_index=edge_index)
        h = h.relu()
        h = self.conv2(h=h, pos=pos, edge_index=edge_index)
        h = h.relu()

        # Global Pooling:
        h = global_max_pool(h, batch)  # [num_examples, hidden_channels]

        # Classifier:
        return self.classifier(h)


model = PointNet()
print(model)

### Tasks
$\rightarrow$ Can you change the PointNet++ module to GCN?<br/>
$\rightarrow$ And to EdgeConv?

**Hint:** You only need to make minimal changes to the `message` function, the type of `aggr` in the `PointNetLayer`, and the input dimensions of the MLP.

## Step 4: Train the model and visualize its outputs.

Now that we have a model and dataloader, we can start optimizing the model. We do this with one of PyTorch's built in optimizers ([Adam](https://arxiv.org/pdf/1412.6980), a gradient descent optimizer using adaptive momentum).

$\rightarrow$ Follow the comments to understand the code.

**Note:** The accuracy will remain quite low ($50-60\%$). This is because the training set is much smaller than usual (only 20 shapes per class, rather than 450) and the model depth is limited. If you want to try a 'full' training loop, delete the `raw` folder in `data` and re-run the notebook, without downloading the smaller dataset.

In [ ]:
# Create the model
model = PointNet().to(device) # Move the model to the GPU if available
# Create the optimizer.
# We let the optimizer know which parameters to optimize and the learning rate (step size).
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# We create the loss function, here a Cross Entropy Loss, typically used for classification tasks.
criterion = torch.nn.CrossEntropyLoss()

# Define the training loop
def train():
    # Set the model to training mode, which enables certain features like dropout, if they are used.
    model.train()

    # Initialize a total loss accumulator for reporting.
    total_loss = 0
    for data in train_loader:
        # First set all the gradients to zero, because we accumulate gradients by default in PyTorch.
        optimizer.zero_grad()
        data = data.to(device)  # Move the data to the GPU if available

        # Forward pass: compute the logits (predictions) for the input data.
        logits = model(data.pos, data.edge_index, data.batch)
        # Compute the loss between the logits and the ground truth labels.
        loss = criterion(logits, data.y)

        # Backward pass: compute the gradients of the loss with respect to the model parameters.
        loss.backward()
        # Update the model parameters using the optimizer.
        optimizer.step()
        # Accumulate the loss for reporting.
        total_loss += float(loss) * data.num_graphs

    return total_loss / len(train_loader.dataset)

# We do not want to compute gradients during testing, so we use torch.no_grad()
# This saves memory and speeds up the computation.
@torch.no_grad()
def test():
    # Set the model to evaluation mode, which disables features like dropout.
    model.eval()

    total_correct = 0
    for data in test_loader:
        data = data.to(device)  # Move the data to the GPU if available
        logits = model(data.pos, data.edge_index, data.batch)
        pred = logits.argmax(dim=-1)
        total_correct += int((pred == data.y).sum())

    return total_correct / len(test_loader.dataset)

# Training loop for 50 epochs
for epoch in tqdm((range(1, 51))):
    loss = train()
    test_acc = test()
    tqdm.write(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Test Acc: {test_acc:.4f}')


### Visualizing the output
To finish up, we show some models and their predicted classes in the test set.

In [ ]:
index = 0
data = test_dataset[index]
data = data.to(device)  # Move the data to the GPU if available
logits = model(data.pos, data.edge_index, data.batch)
pred = logits.argmax(dim=-1)
print(f"Predicted class: {class_names[pred.item()]}, True class: {class_names[data.y.item()]}")
mp.plot(data.pos.cpu().numpy(), shading={"point_size": 0.1})

# Alternatively, visualize the point cloud in polyscope
# ps.init()
# ps.register_point_cloud("ModelNet Object",
#                         data.pos.numpy(), radius=0.0002)
# ps.show()

## Further reading and resources
Check out more tutorials on PyTorch Geometric in the documentation. For example, the tutorial on [Point Cloud Processing](https://pytorch-geometric.readthedocs.io/en/latest/tutorial/point_cloud.html) is quite relevant.

### Advanced tasks to try:
$\rightarrow$ Can you implement a self-attention mechanism within the `MessagePassing` paradigm? If you need inspiration, take a look at some of the convolution layers in PyTorch Geometric (e.g., Graph Attention), but you'll learn more if you first try it yourself.<br />
$\rightarrow$ If you have a mesh with varying triangle densities over the surface (this is the default case), you could weight messages passed from a vertex by the area around that vertex. In geometry processing, we use a mass matrix for this ([LibIGL has a function to compute this for a mesh](https://libigl.github.io/libigl-python-bindings/igl_docs/#massmatrix)). Can you include the mass matrix in the message passing algorithm and train on meshes instead of point clouds?

**Note:** This is not guaranteed to improve or work better, but it's a step in the right direction. If your training data has a consistent connectivity (meshing), your method can overfit on the connectivity. This is highly problematic, because in most real-world applications, the connectivity is not guaranteed and your method will fail.